In [1]:
import pandas as pd
import os

files = os.listdir('./data')
dataframe_list = []
for file in files:
    with open(os.path.join('./data', file)):
        filename = f'{file[0:-4]}_df'
        filename_df = pd.read_csv(f'./data/{file}', encoding="UTF-8")
        dataframe_list.append(filename_df)

In [2]:
print(files)
attFracAllDeaths     = dataframe_list[0]
attFracCancerDeaths  = dataframe_list[1]
attFracLiverCirr     = dataframe_list[2]
attFracTrafficDeaths = dataframe_list[3]
attFracYLL           = dataframe_list[4]
crimePct             = dataframe_list[5]
dependence12moPrev   = dataframe_list[6]
drinkersOnlyConsump  = dataframe_list[7]
harmfulUse12moPrev   = dataframe_list[8]
trafficCrash100k     = dataframe_list[9]
trafficCrashPct      = dataframe_list[10]
useDisorder12moPrex  = dataframe_list[11]

['alc_attrFrac_allDeaths.csv', 'alc_attrFrac_cancerDeaths.csv', 'alc_attrFrac_liverCirr.csv', 'alc_attrFrac_trafficCrashDeath.csv', 'alc_attrFrac_yearsLifeLost.csv', 'alc_crime_percent.csv', 'alc_dependence_12mo_prevalence_percent.xlsx', 'alc_drinkersOnly_consumption_liters_pure_alc.csv', 'alc_harmfulUse_12mo_prevalence_percent.csv', 'alc_trafficCrashes_100k.csv', 'alc_trafficCrashes_percent.csv', 'alc_useDisorder_12mo_prevalence_percent.csv']


In [3]:
print(f'attFracAllDeaths {len(attFracAllDeaths)} \nattFracCancerDeaths {len(attFracCancerDeaths)}')

attFracAllDeaths 582 
attFracCancerDeaths 388


In [4]:
#merge dataframes
joined_df = pd.merge(attFracAllDeaths, attFracCancerDeaths, on = ["Location", "Period", "Dim1"], how = "left", validate = "many_to_many")

#rename
joined_df["AttFracAllDeaths"]    = joined_df["First Tooltip_x"]
joined_df["AttFracCancerDeaths"] = joined_df["First Tooltip_y"]

#drop columns
joined_df = joined_df[["Location", "Period", "Dim1", "AttFracAllDeaths", "AttFracCancerDeaths"]]

####################
#merge liver cirrhosis df
joined_df = pd.merge(joined_df, attFracLiverCirr, on = ["Location", "Period", "Dim1"], how = "left", validate = "many_to_many")
#rename
joined_df["AttFracLiverCirr"] = joined_df["First Tooltip"]
#drop columns
joined_df = joined_df[["Location", "Period", "Dim1", "AttFracAllDeaths", "AttFracCancerDeaths", "AttFracLiverCirr"]]

###########################
#merge traffic deaths df
joined_df = pd.merge(joined_df, attFracTrafficDeaths, on = ["Location", "Period", "Dim1"], how = "left", validate = "many_to_many")
#rename
joined_df["AttFracTrafficDeaths"] = joined_df["First Tooltip"]
#drop columns
joined_df = joined_df[["Location", "Period", "Dim1", "AttFracAllDeaths", "AttFracCancerDeaths", "AttFracLiverCirr", "AttFracTrafficDeaths"]]

#################
#merge YLL df
attFracYLL["Dim1"] = "Both sexes"
joined_df = pd.merge(joined_df, attFracYLL, on = ["Location", "Period", "Dim1"], how = "left", validate = "many_to_many")
#rename
joined_df["AttFracYLL"] = joined_df["First Tooltip"]
#drop columns
joined_df = joined_df[["Location", "Period", "Dim1", "AttFracAllDeaths", "AttFracCancerDeaths", "AttFracLiverCirr", "AttFracTrafficDeaths","AttFracYLL"]]

###########################
#merge dependence prev
joined_df = pd.merge(joined_df, dependence12moPrev, on = ["Location", "Period", "Dim1"], how = "left", validate = "many_to_many")
#rename
joined_df["Dependence12MoPrev"] = joined_df["First Tooltip"]
#drop columns
joined_df = joined_df[["Location", "Period", "Dim1", "AttFracAllDeaths", "AttFracCancerDeaths", "AttFracLiverCirr", "AttFracTrafficDeaths","AttFracYLL", "Dependence12MoPrev"]]

################################
#clean drinkers only consumption
new = drinkersOnlyConsump["First Tooltip"].str.split(" ", n = 1, expand = True) 
  
# making separate first name column from new data frame 
drinkersOnlyConsump["DrinkersOnlyLitersAlc"]= new[0] 
  
# making separate last name column from new data frame 
drinkersOnlyConsump["DrinkersOnlyCI"]= new[1] 
  
# Dropping old Name columns 
drinkersOnlyConsump.drop(columns =["First Tooltip"], inplace = True) 

#merge drinkers only consumption
joined_df = pd.merge(joined_df, drinkersOnlyConsump, on = ["Location", "Period", "Dim1"], how = "left", validate = "many_to_many")
#drop columns
joined_df = joined_df[["Location", "Period", "Dim1", "AttFracAllDeaths", "AttFracCancerDeaths", "AttFracLiverCirr", "AttFracTrafficDeaths","AttFracYLL", "Dependence12MoPrev", "DrinkersOnlyLitersAlc", "DrinkersOnlyCI"]]

########################
#merge harmful use prev
joined_df = pd.merge(joined_df, harmfulUse12moPrev, on = ["Location", "Period", "Dim1"], how = "left", validate = "many_to_many")
#rename
joined_df["HarmfulUse12MoPrev"] = joined_df["First Tooltip"]
#drop columns
joined_df = joined_df[["Location", "Period", "Dim1", "AttFracAllDeaths", "AttFracCancerDeaths", "AttFracLiverCirr", "AttFracTrafficDeaths","AttFracYLL", "Dependence12MoPrev", "DrinkersOnlyLitersAlc", "DrinkersOnlyCI", "HarmfulUse12MoPrev"]]

In [5]:
#merge trafficCrash100k df
trafficCrash100k["Dim1"] = "Both sexes"
joined_df = pd.merge(joined_df, trafficCrash100k, on = ["Location", "Period", "Dim1"], how = "outer", validate = "many_to_many")
#rename
joined_df["TrafficCrash100k"] = joined_df["First Tooltip"]
joined_df = joined_df[["Location", "Period", "Dim1", "AttFracAllDeaths", "AttFracCancerDeaths", "AttFracLiverCirr", "AttFracTrafficDeaths","AttFracYLL", "Dependence12MoPrev", "DrinkersOnlyLitersAlc", "DrinkersOnlyCI", "HarmfulUse12MoPrev", "TrafficCrash100k"]]

#check the merge for extra years
pd.crosstab(joined_df.Location, joined_df.Period)

joined_df.sort_values(by =["Location","Period"]).head(50)

########################
#merge trafficCrash PCT df
trafficCrashPct["Dim1"] = "Both sexes"
joined_df = pd.merge(joined_df, trafficCrashPct, on = ["Location", "Period", "Dim1"], how = "outer", validate = "many_to_many")
#rename
joined_df["TrafficCrashPct"] = joined_df["First Tooltip"]
joined_df = joined_df[["Location", "Period", "Dim1", "AttFracAllDeaths", "AttFracCancerDeaths", "AttFracLiverCirr", "AttFracTrafficDeaths","AttFracYLL", "Dependence12MoPrev", "DrinkersOnlyLitersAlc", "DrinkersOnlyCI", "HarmfulUse12MoPrev", "TrafficCrash100k", "TrafficCrashPct"]]

#check the merge for extra years
pd.crosstab(joined_df.Location, joined_df.Period)


########################
#merge useDisorder12moPrex
joined_df = pd.merge(joined_df, useDisorder12moPrex, on = ["Location", "Period", "Dim1"], how = "outer", validate = "many_to_many")
#rename
joined_df["UseDisorder12moPrev"] = joined_df["First Tooltip"]
joined_df = joined_df[["Location", "Period", "Dim1", "AttFracAllDeaths", "AttFracCancerDeaths", "AttFracLiverCirr", "AttFracTrafficDeaths","AttFracYLL", "Dependence12MoPrev", "DrinkersOnlyLitersAlc", "DrinkersOnlyCI", "HarmfulUse12MoPrev","UseDisorder12moPrev", "TrafficCrash100k", "TrafficCrashPct"]]

#reset index
joined_df.sort_values(by=['Location', 'Period'], inplace= True)
joined_df.reset_index()

,index,Location,Period,Dim1,AttFracAllDeaths,AttFracCancerDeaths,AttFracLiverCirr,AttFracTrafficDeaths,AttFracYLL,Dependence12MoPrev,DrinkersOnlyLitersAlc,DrinkersOnlyCI,HarmfulUse12MoPrev,UseDisorder12moPrev,TrafficCrash100k,TrafficCrashPct
0,0,Afghanistan,2016,Both sexes,0.2,NaN,NaN,NaN,1,0.2,26.9,[24.4-29.3],0.2,0.4,NaN,NaN
1,1,Afghanistan,2016,Male,0.3,0.5,5.5,0.7,NaN,0.3,30.4,[27.7-33.2],0.3,0.6,NaN,NaN
2,2,Afghanistan,2016,Female,0.1,0.1,1.1,0.5,NaN,0.1,11.5,[10.4-12.5],0,0.1,NaN,NaN
3,602,Albania,1995,Both sexes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,3.3
4,601,Albania,1996,Both sexes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1634,577,Zambia,2016,Male,6.4,3.4,57.4,25.9,NaN,3.3,22.2,[21-23.3],6.5,9.8,NaN,NaN
1635,578,Zambia,2016,Female,1.6,0.8,31.9,18.6,NaN,0.5,8.4,[7.9-8.8],0.7,1.2,NaN,NaN
1636,579,Zimbabwe,2016,Both sexes,3.1,NaN,NaN,NaN,4,2.2,26.4,[25.4-27.5],4.2,6.4,NaN,NaN
1637,580,Zimbabwe,2016,Male,4.9,2.7,54.4,17.1,NaN,3.8,32.2,[31-33.5],7.3,11.1,NaN,NaN


In [6]:
#crime pct

crimePct["Location"] = crimePct["First Location"]
crimePct["Period"]   = crimePct["First Period"]
crimePct["CrimePct"] = crimePct["First Tooltip"]
crimePct["Dim1"] = "Both sexes"

crimePct = crimePct[["GeometryCode", "Location", "Period", "CrimePct", "Dim1"]]

#merge
joined_df = pd.merge(joined_df, crimePct, on = ["Location", "Period", "Dim1"], how = "outer", validate = "many_to_one")
#rename
joined_df = joined_df[["Location", "Period", "Dim1", "AttFracAllDeaths", "AttFracCancerDeaths", "AttFracLiverCirr", "AttFracTrafficDeaths","AttFracYLL", "Dependence12MoPrev", "DrinkersOnlyLitersAlc", "DrinkersOnlyCI", "HarmfulUse12MoPrev","UseDisorder12moPrev", "TrafficCrash100k", "TrafficCrashPct","GeometryCode", "CrimePct"]]


In [7]:
joined_df.head(500)

,Location,Period,Dim1,AttFracAllDeaths,AttFracCancerDeaths,AttFracLiverCirr,AttFracTrafficDeaths,AttFracYLL,Dependence12MoPrev,DrinkersOnlyLitersAlc,DrinkersOnlyCI,HarmfulUse12MoPrev,UseDisorder12moPrev,TrafficCrash100k,TrafficCrashPct,GeometryCode,CrimePct
0,Afghanistan,2016,Both sexes,0.2,NaN,NaN,NaN,1,0.2,26.9,[24.4-29.3],0.2,0.4,NaN,NaN,NaN,NaN
1,Afghanistan,2016,Male,0.3,0.5,5.5,0.7,NaN,0.3,30.4,[27.7-33.2],0.3,0.6,NaN,NaN,NaN,NaN
2,Afghanistan,2016,Female,0.1,0.1,1.1,0.5,NaN,0.1,11.5,[10.4-12.5],0,0.1,NaN,NaN,NaN,NaN
3,Albania,1995,Both sexes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,3.3,NaN,NaN
4,Albania,1996,Both sexes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3,1.8,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,El Salvador,2016,Female,1.9,1,27.8,17.4,NaN,1.8,6,[5.8-6.3],1.8,3.6,NaN,NaN,NaN,NaN
496,Equatorial Guinea,2016,Both sexes,9.4,NaN,NaN,NaN,5,2.4,16.3,[15.8-16.7],4.4,6.8,NaN,NaN,NaN,NaN
497,Equatorial Guinea,2016,Male,13.1,8.6,72.4,58.6,NaN,3.8,21.7,[21.1-22.4],7.4,11.2,NaN,NaN,NaN,NaN
498,Equatorial Guinea,2016,Female,4.2,2.5,57.7,53.2,NaN,0.8,8,[7.7-8.2],1.2,2,NaN,NaN,NaN,NaN


In [9]:
joined_df.to_csv("who_alchohol_data.csv", index=False)